### LASSO regression

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

import sqlite3
conn=sqlite3.connect("parametres.db")
cur=conn.cursor()

In [2]:
df=pd.read_csv('delayed25.csv', index_col=False, warn_bad_lines=True, error_bad_lines=False)
# as the process take too much time, we resample the database
df = df.sample(frac =.1)
df=df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48419 entries, 323984 to 78907
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ActualElapsedTime  48419 non-null  int64  
 1   AirTime            48419 non-null  int64  
 2   ArrDelay           48419 non-null  int64  
 3   DepDelay           48419 non-null  float64
 4   Distance           48419 non-null  float64
 5   TaxiIn             48419 non-null  float64
 6   TaxiOut            48419 non-null  float64
 7   Cancelled          48419 non-null  float64
 8   Diverted           48419 non-null  int64  
 9   CarrierDelay       48419 non-null  float64
 10  WeatherDelay       48419 non-null  float64
 11  NASDelay           48419 non-null  float64
 12  SecurityDelay      48419 non-null  float64
 13  LateAircraftDelay  48419 non-null  float64
 14  DepTime_sin        48419 non-null  float64
 15  DepTime_cos        48419 non-null  float64
 16  Month_sin        

In [3]:
X,y = df.loc[:, df.columns != 'ArrDelay'],df.loc[:, df.columns == 'ArrDelay']

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [4]:
Xdata = X.values
ydata = y.values

In [5]:
from numpy import absolute
from numpy import mean
from numpy import std

In [ ]:
# looking for the optimal aplpha value

list=[20,10,5,1,0.1,0.01,0.001,0.0001]
for val in list:
    model = Lasso(alpha=val)
    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model with CROSS VALIDATION
    scores = cross_val_score(model, Xdata, ydata, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    # force scores to be positive
    scores = absolute(scores)
    print('for aplha=',val,'  Mean MAE: %.3f' % (mean(scores)))

    
    

for aplha= 20   Mean MAE: 9.106
for aplha= 10   Mean MAE: 5.775
for aplha= 5   Mean MAE: 4.285
for aplha= 1   Mean MAE: 3.891


In [6]:
# Use the best alpha 
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train, y_train) 

/Users/psans/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 582273.7448225775, tolerance: 12849.588914588874
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.01)

In [11]:

pred_train_lasso= model_lasso.predict(X_train)
pred_test_lasso= model_lasso.predict(X_test)




y_pred = model_lasso.predict(X_test)

model_lasso.fit(X,y)
print("R-Squared on train dataset={}".format(model_lasso.score(X_test,y_test)))
r2=model_lasso.score(X_test,y_test)


mse = mean_squared_error(y_test,pred_test_lasso)
print('MSE: %.3f' % mse)
print('RMSE: %.3f' % (mse*(1/2.0)))

mae=mean_absolute_error(y_test,pred_test_lasso)
print('MAE: %.3f' % mae)



R-Squared on train dataset=0.9867742229303337
MSE: 44.190
RMSE: 22.095
MAE: 3.548


/Users/psans/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 800892.9602510676, tolerance: 16085.266240760864
  model = cd_fast.enet_coordinate_descent(


In [13]:
#SQL database update
code='ls1'
process='Lasso'
datab='standard'
depdelay='yes'

cur.execute("INSERT OR REPLACE INTO parametres(codi,proces,DepDelay, base_dades, MAE, R2, MSE) VALUES(?,?,?,?,?,?,?)",(code,process,depdelay,datab,'%.3f' % mae,'%.3f' % r2,'%.3f' % mse))
conn.commit()

##### Eliminem la columna DepDelay

In [14]:
dfdel=df.drop(['DepDelay'], axis=1)

In [15]:
X,y = dfdel.loc[:, dfdel.columns != 'ArrDelay'],dfdel.loc[:, dfdel.columns == 'ArrDelay']

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

Xdata = X.values
ydata = y.values

In [16]:
model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train, y_train) 

pred_train_lasso= model_lasso.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
print(r2_score(y_train, pred_train_lasso))


pred_test_lasso= model_lasso.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print(r2_score(y_test, pred_test_lasso))

6.0519530288049435
0.9889590759160927
8.124146862371434
0.9802462238297284


/Users/psans/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 713686.4653260664, tolerance: 12849.588914588874
  model = cd_fast.enet_coordinate_descent(


In [17]:
#SQL database update
code='ls2'
process='Lasso'
datab='standard'
depdelay='no'

cur.execute("INSERT OR REPLACE INTO parametres(codi,proces,DepDelay, base_dades, MAE, R2, MSE) VALUES(?,?,?,?,?,?,?)",(code,process,depdelay,datab,'%.3f' % mae,'%.3f' % r2,'%.3f' % mse))
conn.commit()

In [18]:
print(pd.read_sql_query("SELECT * from parametres", conn))

  codi          proces DepDelay    base_dades     MAE     R2      MSE
0  dt1   Decision_Tree      yes      standard   5.826  0.977   83.016
1  dt2   Decision_Tree      yes      standard   9.383  0.936  205.708
2  dt3   Decision_Tree       no      standard  15.117  0.840  524.455
3  dt4   Decision_Tree      yes  standard/min   9.383  0.840  205.708
4  dt5   Decision_Tree      yes   no standard   9.818  0.840  227.035
5  dt6   Decision_Tree       no   no standard  14.717  0.840  493.721
6  nn1  Neural_network      yes      standard   2.295  0.995   16.489
7  nn2  Neural_network       no      standard   2.822  0.989   35.978
8  ls1           Lasso      yes      standard   3.548  0.987   44.190
9  ls2           Lasso       no      standard   3.548  0.987   44.190
